In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import sys
import datetime


sys.path.append('..')
from utils.tte import tte


DATA_DIR = Path("../data/cl/processed") 

rolled_raw = pd.read_csv(DATA_DIR / "cl_front_month_raw.csv")
panama_rolled = pd.read_csv(DATA_DIR / "cl_front_month_panama_adjusted.csv")
proportional_rolled = pd.read_csv(DATA_DIR / "cl_front_month_proportional_adjusted.csv")

spreads = pd.read_csv(DATA_DIR / "cl_spreads.csv")


In [ ]:
raw = pd.read_csv("../data/cl/processed/cl_raw.csv", index_col=0, parse_dates=True)
raw[["letters", "number"]] = raw["symbol"].str.extract(r"([a-zA-Z]+)([0-9]+)")
raw["number"] = raw["number"].astype(int)
raw = raw.sort_values(by=["ts_event", "number", "letters"]).reset_index()
raw["month"] = raw["letters"].str[-1]
raw = raw.drop(columns=["letters"])

In [ ]:
raw[["ts_event", "symbol", "volume", "month", "number"]]

In [ ]:
raw["expiration_year"] = np.where((raw["ts_event"].dt.year//10 *10)+raw["number"]<raw["ts_event"].dt.year,((raw["ts_event"].dt.year//10)+1)*10,(raw["ts_event"].dt.year//10)*10) + raw["number"] #type: ignore

In [ ]:
month_codes = "FGHJKMNQUVXZ"

conversions = {
    "F": "-01-20",
    "G": "-02-20",
    "H": "-03-20",
    "J": "-04-20",
    "K": "-05-20",
    "M": "-06-20",
    "N": "-07-20",
    "Q": "-08-20",
    "U": "-09-20",
    "V": "-10-20",
    "X": "-11-20",
    "Z": "-12-20"
}

#pattern = rf'^CL[{month_codes}]\d$'

raw["expiration_date"] = pd.to_datetime((raw["expiration_year"].astype(str) + raw["month"].map(conversions))) #type: ignore
raw["expiration_date"] = raw["expiration_date"].dt.tz_localize("UTC") #type: ignore
raw


In [ ]:
raw["tte"] = (raw["expiration_date"] - raw["ts_event"]).dt.days / 365.25 #type: ignore

In [ ]:
#raw = raw.drop(columns=["number", "month", "expiration_year"])
spreads[["ts_event", "symbol", "close", "volume"]]


In [ ]:
first_month = rf'^CL[{month_codes}]\d$'
second_month = rf'^CL[{month_codes}]\d$'


#spreads[["letters"]] = spreads["symbol"].str.extract() #type: ignore
spreads[["month_from", "year_from", "month_to", "year_to"]] = spreads["symbol"].str.extract(rf"CL([{month_codes}])([0-9])-CL([{month_codes}])([0-9])")
spreads = spreads[["ts_event", "symbol", "close", "volume", "month_from", "year_from", "month_to", "year_to"]]
spreads["year_from"] = spreads["year_from"].astype(int)
spreads["year_to"] = spreads["year_to"].astype(int)
spreads["ts_event"] = pd.to_datetime(spreads["ts_event"])

In [ ]:

#spreads["from_date"] = pd.to_datetime(((np.where((spreads["ts_event"].dt.year//10 *10)+spreads["year_from"]<spreads["ts_event"].dt.year,2020,2010) + spreads["year_from"]).astype(str)+ spreads["month_from"].map(conversions))).dt.tz_localize('UTC') #type: ignore
#spreads["to_date"] = pd.to_datetime(((np.where((spreads["ts_event"].dt.year//10 *10)+spreads["year_to"]<spreads["ts_event"].dt.year,2020,2010) + spreads["year_to"]).astype(str)+ spreads["month_to"].map(conversions))).dt.tz_localize('UTC') #type: ignore
#
#spreads["year"] = spreads["ts_event"].dt.year//10 * 10 #type: ignore
#spreads["number"] = spreads["year_from"]
#spreads["test_year"] = spreads["year"] + spreads["number"]
#spreads["test2"] = np.where(spreads["test_year"]<spreads["ts_event"].dt.year, spreads["test_year"]+10, spreads["test_year"]) #type: ignore
#spreads[["ts_event", "tte", "year", "year_from", "test_year", "test2", "from_date"]]


In [ ]:


spreads["from_date"] = pd.to_datetime(((np.where((spreads["ts_event"].dt.year//10 *10)+spreads["year_from"]<spreads["ts_event"].dt.year,((spreads["ts_event"].dt.year//10)+1)*10,(spreads["ts_event"].dt.year//10)*10) + spreads["year_from"]).astype(str)+ spreads["month_from"].map(conversions))).dt.tz_localize('UTC') #type: ignore
spreads["to_date"] = pd.to_datetime(((np.where((spreads["ts_event"].dt.year//10 *10)+spreads["year_to"]<spreads["ts_event"].dt.year,((spreads["ts_event"].dt.year//10)+1)*10,(spreads["ts_event"].dt.year//10)*10) + spreads["year_to"]).astype(str)+ spreads["month_to"].map(conversions))).dt.tz_localize('UTC') #type: ignore



spreads["tenor"] = (spreads["to_date"] - spreads["from_date"]).dt.days / 365.25 #type: ignore
spreads["tte"] = (spreads["to_date"] - spreads["ts_event"]).dt.days / 365.25 #type: ignore

spreads.sort_values(by=["ts_event", "year_from", "month_from", "tenor"], inplace=True)

spreads.reset_index(drop=True, inplace=True)
spreads


In [ ]:
raw = raw[raw["ts_event"] >= pd.to_datetime("2011-11-27").tz_localize("UTC")].reset_index(drop=True)
raw[["ts_event", "symbol", "close", "volume", "month", "expiration_date", "tte"]]

In [ ]:
df = raw.loc[raw["ts_event"] >= pd.to_datetime("2011-11-27").tz_localize("UTC")]
df["month_to"] = df["month"]
df["year_to"] = df["number"]
df = df.rename(columns={"month": "month_from", "number": "year_from"})
df = df[["ts_event", "symbol", "close", "volume", "month_from",  "year_from", "month_to", "year_to", "tte"]]
df


In [ ]:
dfs = spreads.loc[spreads["ts_event"] >= pd.to_datetime("2011-11-27").tz_localize("UTC")]
dfs = dfs[["ts_event", "symbol", "close", "volume", "month_from", "year_from", "month_to", "year_to", "tte"]]
#dfs = dfs.rename(columns={"month_to": "month", "year_to": "number"})
dfs

In [ ]:
target = pd.concat([df, dfs], ignore_index=True)
target.sort_values(by=["tte", "volume"], inplace=True, ascending=[True, False])
target.reset_index(drop=True, inplace=True)
target = target.groupby(["ts_event", "tte"]).first().reset_index()
#target[["ts_event", "symbol", "close", "volume", "tte"]]
target

In [ ]:
#target = target[target["ts_event"] == pd.to_datetime("2017-11-08").tz_localize("UTC")]

target["year_from"] =  np.where(((target["ts_event"].dt.year//10 * 10)+target["year_from"])<target["ts_event"].dt.year, target["year_from"]+10, target["year_from"]) #type: ignore
target["year_to"] =  np.where(((target["ts_event"].dt.year//10 * 10)+target["year_to"])<target["ts_event"].dt.year, target["year_to"]+10, target["year_to"]) #type: ignore

target

In [ ]:
m_map = {'F':1, 'G':2, 'H':3, 'J':4, 'K':5, 'M':6, 'N':7, 'Q':8, 'U':9, 'V':10, 'X':11, 'Z':12}
#target["month_from_num"] = target["month_from"].map(m_map) 
#target["month_to_num"] = target["month_to"].map(m_map).fillna(0).astype(int) + target["year_to"]*12
#
#target["abs_from"] = target["month_from_num"] + target["year_from"]*12
#target["abs_to"] = target["month_to_num"] + target["year_to"]*12
#
#start_month = target["abs_from"].min()
#target["idx_from"] = target["abs_from"] - start_month
#target["idx_to"] = target["abs_to"] - start_month
target["position_from"] = target["month_from"].map(m_map) + target["year_from"]*12
target["position_to"] = target["month_to"].map(m_map) + target["year_to"]*12
start_month = target["position_from"].min()
target["position_from"] = target["position_from"] - start_month
target["position_to"] = target["position_to"] - start_month



target



In [ ]:
row_indices = np.arange(len(target))
A = np.zeros((len(target), target["position_to"].max()+1))
A

In [ ]:
A[row_indices, target["position_to"]] = -1
A[row_indices, target["position_from"]] = 1
A


In [ ]:
x, residuals, rank, s = np.linalg.lstsq(A, target["close"], rcond=None)
x

In [ ]:
result = pd.DataFrame()
result["date"] = target["ts_event"]
result["symbol"] = "CL" + target["month_to"] + (target["year_to"] % 10).astype(str)
result["close"] = np.round(x[target["position_to"]], 3)
result["tte"] = target["tte"]
result

In [ ]:
from cProfile import label
import matplotlib 

slice = result[result["date"] == pd.to_datetime("2020-11-08").tz_localize("UTC")]
plt.plot(slice["tte"], slice["close"], label='Raw Forward Curve', marker='o')
plt.title('WTI Crude Forward Curve on 2017-11-08')
plt.xlabel('Time to Expiration (Years)')
plt.ylabel('Price (USD)')
fig = plt.gcf()
fig.set_size_inches(12, 6)
fig.set_dpi(600)
plt.tight_layout()
plt.legend()
plt.show()


In [ ]:
def nelson_siegel(tau, beta0, beta1, beta2, lambda_):
    term1 = (1 - np.exp(-tau / lambda_)) / (tau / lambda_)
    term2 = term1 - np.exp(-tau / lambda_)
    return beta0 + beta1 * term1 + beta2 * term2

def objective(x, tau, close):
    beta0, beta1, beta2, lambda_ = x
    prices = nelson_siegel(tau, beta0, beta1, beta2, lambda_)
    loss = np.sum((prices - close) ** 2)
    return loss

In [ ]:
from scipy.optimize import minimize
from joblib import Parallel, delayed


params = pd.DataFrame(columns=["date", "beta0", "beta1", "beta2", "lambda"])

bounds = [(1e-6, None), (None, None), (None, None), (0.01, 10.0)]

temp_results = []


def solve_ns_row(date,row):
    tau = row["tte"].values
    close = row["close"].values
    x0 = [row["close"].iloc[-1], row["close"].iloc[0]-row["close"].iloc[-1], 0, 1.0]

    res = minimize(objective, x0, args=(tau, close), method='L-BFGS-B', bounds=bounds)
    beta0, beta1, beta2, lambda_ = res.x

    return {"date": date, "beta0": beta0, "beta1": beta1, "beta2": beta2, "lambda": lambda_}

temp_results = Parallel(n_jobs=-1)(delayed(solve_ns_row)(date, row) for date, row in result.groupby("date"))

#for date, row in result.groupby("date"):
#    tau = row["tte"].values
#    close = row["close"].values
#    x0 = [row["close"].iloc[-1], row["close"].iloc[0]-row["close"].iloc[-1], 0, 1.0]
#
#    res = minimize(objective, x0, args=(tau, close), method='L-BFGS-B', bounds=bounds)
#    beta0, beta1, beta2, lambda_ = res.x
#
#    temp_results.append({"date": date, "beta0": beta0, "beta1": beta1, "beta2": beta2, "lambda": lambda_})
#
params = pd.DataFrame(temp_results)
params

    
    
    

In [ ]:
params.to_csv(DATA_DIR / "forward_curve_ns_params.csv", index=False)

In [ ]:
result = pd.merge(result, params, on='date', how='left')

In [ ]:
result

In [ ]:
result["nelson_siegel"] = nelson_siegel(result["tte"],
    result["beta0"],
    result["beta1"],
    result["beta2"],
    result["lambda"],
)
result["loss"] = (result["close"] - result["nelson_siegel"])**2
loss = result["loss"].mean()
print(f"Mean Squared Error of Nelson-Siegel Fit: {loss}")
mean = result["close"].mean()
print(f"This is off by {loss/mean:.4%} of the average price.")

In [ ]:
result